In [ ]:
# !pip install llama_index==0.11.4
# !pip install PyYAML
# !pip install plotly==5.24.0
# !pip install docx2txt==0.8
# !pip install chromadb==0.5.5
# !pip install llama-index-vector-stores-chroma==0.2.0
# !pip install llama-index-extractors-entity==0.2.0
# !pip install llama-index-readers-web==0.2.2
# !pip install transformers==4.40.2

# Llama Index cơ bản

Trong phần này, chúng ta sẽ tìm hiểu kiến trúc của một hệ thống RAG cơ bản được xây dựng bởi Llama index sẽ gồm các thành phần nào. Bạn có thể tạo một ứng dụng RAG cơ bản sau khi đọc xong phần này.

## Documents

Một thành phần không thể thiếu trong các ứng dụng RAG là dữ liệu, chúng ta có nhiều loại dữ liệu khác nhau như: pdf, docx, csv, pptx, html, database... mỗi loại khác nhau không tuân theo một quy chuẩn chung nào. Chính vì vậy, Documents trong Llama Index được tạo ra với vai trò như một cái khuôn để chứa và đưa các loại dữ liệu về một loại cấu trúc chung.

Điểm đặc biệt trong cấu trúc của Documents là nó có thể chứa thêm các thông tin bổ xung về dữ liệu trong đó, thành phần này gọi là siêu dữ liệu(metadata). Hiểu đơn giản thì nó là các thông tin bổ xung mà chúng ta cung cấp thêm cho dữ liệu như mô tả, tóm tắt, tiêu đề... những thông tin này nhằm mục đích hỗ trợ quá trình tìm kiếm hiệu quả hơn.

Để tạo Document từ dữ liệu, cú pháp sử dụng như sau: `Document(text, metadata, id_)`
Trong đó:

    * text: dữ liệu đầu vào dạng text

    * metadata: siêu dữ liệu, thường có định dạng dictionary, thường được tạo tự động bởi thư viện.

    * id_: Chỉ số của đối tượng Document này, nó thường được tạo một cách tự động


In [3]:
from llama_index.core import Document

text = "AI VIET NAM"
doc = Document(
        text=text,
        metadata={"fb": "fb/aivietnam.edu.vn"},
        id_="1"
        )
print(doc)

Doc ID: 1
Text: AI VIET NAM


In [4]:
for i in doc:
    print(i)

('id_', '1')
('embedding', None)
('metadata', {'fb': 'fb/aivietnam.edu.vn'})
('excluded_embed_metadata_keys', [])
('excluded_llm_metadata_keys', [])
('relationships', {})
('text', 'AI VIET NAM')
('mimetype', 'text/plain')
('start_char_idx', None)
('end_char_idx', None)
('text_template', '{metadata_str}\n\n{content}')
('metadata_template', '{key}: {value}')
('metadata_seperator', '\n')


Trong ví dụ trên, chúng ta tạo một đối tượng Document với đầu vào là text-một chuỗi string, và siêu dữ liệu metadata là một dictionary 

## Nodes

Việc tạo Documents khá đơn giản, tuy nhiên dữ liệu trong document vẫn là dữ liệu thô, vậy làm sao để  dữ liệu thô có thể chuyển đổi thành định dạng mà LLM có thể xử lý và suy luận hiệu quả. Giải pháp chính là Nodes, nó là những nội dung nhỏ hơn được trích xuất từ tài liệu, mục đích để chia tài liệu thành những phần nhỏ hơn để dễ quản lí.

Nodes tránh được tình trạng vượt quá giới hạn của prompt mà mô hình cho phép. Ví dụ khi có 1 cuốn ebook 100 trang chúng ta không dùng toàn bộ dữ liệu trong đó trực tiếp vào prompt để LLM xử lý, vì nó sẽ vượt quá giới hạn đầu vào của mô hình. Hơn nữa cách này có nhược điểm là chi phí xử lí cao, chúng ta phải trả nhiều tiền hơn cho việc sử dụng API, độ chính xác của câu trả lời cũng không đảm bảo vì khi đầu vào là một văn bản dài như vậy, prompt của chúng ta không tập trung vào thông tin cụ thể nào dẫn đến mô hình không hiểu và đưa ra câu trả lời chính xác. Ngoài ra giữa các nodes chúng ta có thể thiết lập mối quan hệ giữa chúng.

Trong Llamaindex, chúng ta có thể tạo nodes cho dữ liệu văn bản bằng cách sử dụng lớp TextNode. Cú pháp sử dụng:



In [5]:
from llama_index.core import Document
from llama_index.core.schema import TextNode

text = "AI VIET NAM"
doc = Document(
        text=text
        )

node1 = TextNode(text=doc.text[:2])
node2 = TextNode(text=doc.text[3:])

print(node1)
print(node2)

Node ID: 84f66afa-d8b1-4edf-9658-d3e44a746fab
Text: AI
Node ID: 7edd07b6-27dc-4ba4-9a84-a0b5e9ea200b
Text: VIET NAM


Chúng ta có thể tạo các node tự động bằng cách sử dụng TokenTextSplitter, cú pháp sử dụng:

```Python
from llama_index.core.node_parser import TokenTextSplitter

TokenTextSplitter(
    chunk_size,
    chunk_overlap,
    separator,
)
```

Trong đó: 

    - chunk_size: Kích thước của mỗi đoạn văn bản (chunk). Đây là số lượng token (từ hoặc ký tự) trong mỗi đoạn.

    - chunk_overlap: Số lượng token sẽ bị trùng lặp giữa các đoạn liên tiếp. Điều này có nghĩa là một phần của đoạn trước sẽ được lặp lại ở đoạn sau.
    
    - separator: Ký tự hoặc chuỗi ký tự được sử dụng để phân tách các đoạn văn bản. 


In [6]:
from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter

text = "Hôm nay trời nắng, tôi đi ăn kem, lạnh buốt cả răng!"
doc = Document(text=text)
splitter = TokenTextSplitter(
    chunk_size=20,
    chunk_overlap=5,
    separator= " "
)
nodes = splitter.get_nodes_from_documents([doc])

for node in nodes:
    print(node)

Metadata length (2) is close to chunk size (20). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Node ID: 38081e25-27e8-4152-8da3-47627107ae55
Text: Hôm nay trời nắng, tôi đi ăn
Node ID: d5c34421-6eee-49b3-a7a4-b79de4afeb0d
Text: đi ăn kem, lạnh buốt cả răng!


Trong ví dụ trên, chúng ta gọi lớp TokenTextSplitter từ module node_parser trong gói llama_index.core. Tiếp theo, chúng ta tạo đối tượng doc từ text, sau đó 
tạo đối tượng splitter từ lớp TokenTextSplitter với các thuộc tính chunk_size, chunk_overlap, separator. Để tạo node, chúng ta gọi đến phương thức get_nodes_from_documents từ đối tượng splitter vừa tạo. Vậy là các node của chúng ta đã được tạo tự động. Một cảnh báo xuất hiện nói rằng chúng ta sẽ gặp vấn đề khi mà chunk_size của chúng ta quá nhỏ, nhỏ hơn hoặc chênh lệch không quá nhiều so với kích cỡ của metadata. Vì khi đó metadata sẽ chiếm phần lớn lượng dữ liệu của node trong khi dữ liệu thực tế  thì rất ít.

Việc tạo node không chỉ dừng lại ở việc chia nhỏ document, chúng ta có thể thiết lập mối quan hệ giữa các node, ngoài ra còn có một số phương pháp biến đổi node nâng cao hơn mà chúng ta sẽ tìm hiểu ở phần sau. Dưới đây là cách mà chúng ta tạo mối quan hệ giữa hai node, có 5 mối quan hệ giữa các node là:
Trong hệ thống phân chia văn bản thành các đoạn nhỏ (nodes) như được sử dụng trong thư viện `llama_index`, các mối quan hệ giữa các nodes có thể được mô tả như sau:

1. **SOURCE**: 
   - Node này là tài liệu gốc (source document). Đây là toàn bộ văn bản ban đầu mà từ đó các nodes khác được tách ra.
   - Ví dụ: Nếu văn bản gốc là "Hôm nay trời nắng, tôi đi ăn kem, lạnh buốt cả răng!", thì node `SOURCE` sẽ chứa toàn bộ văn bản này.

2. **PREVIOUS**: 
   - Node này là node trước đó trong tài liệu. Nó đại diện cho đoạn văn bản ngay trước node hiện tại.
   - Ví dụ: Nếu có hai nodes liên tiếp, node thứ hai sẽ có một liên kết `PREVIOUS` trỏ đến node thứ nhất.

3. **NEXT**: 
   - Node này là node kế tiếp trong tài liệu. Nó đại diện cho đoạn văn bản ngay sau node hiện tại.
   - Ví dụ: Nếu có hai nodes liên tiếp, node thứ nhất sẽ có một liên kết `NEXT` trỏ đến node thứ hai.

4. **PARENT**: 
   - Node này là node cha trong tài liệu. Nó đại diện cho một đoạn văn bản lớn hơn chứa node hiện tại.
   - Ví dụ: Nếu một tài liệu lớn được chia thành các đoạn nhỏ hơn và mỗi đoạn nhỏ này lại được chia thành các đoạn nhỏ hơn nữa, thì một node có thể có một node cha chứa nó.

5. **CHILD**: 
   - Node này là node con trong tài liệu. Nó đại diện cho một đoạn văn bản nhỏ hơn nằm trong node hiện tại.
   - Ví dụ: Nếu một đoạn văn bản lớn được chia thành các đoạn nhỏ hơn, thì mỗi đoạn nhỏ sẽ là một node con của node lớn.

In [7]:
print(doc)
print(nodes[0])
print(nodes[1])

Doc ID: 2fa6a6a0-3911-4743-aa53-a4e7032c5607
Text: Hôm nay trời nắng, tôi đi ăn kem, lạnh buốt cả răng!
Node ID: 38081e25-27e8-4152-8da3-47627107ae55
Text: Hôm nay trời nắng, tôi đi ăn
Node ID: d5c34421-6eee-49b3-a7a4-b79de4afeb0d
Text: đi ăn kem, lạnh buốt cả răng!


In [8]:
nodes

[TextNode(id_='38081e25-27e8-4152-8da3-47627107ae55', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2fa6a6a0-3911-4743-aa53-a4e7032c5607', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='0af2325bc3976ef85024e35769344ac7138fe4c21700c2bf55d2109039882bb4'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='d5c34421-6eee-49b3-a7a4-b79de4afeb0d', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='524e34237455ee85fecf2724237a8e825e136e9dff359bbde44c4c3a656c1d95')}, text='Hôm nay trời nắng, tôi đi ăn', mimetype='text/plain', start_char_idx=0, end_char_idx=28, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='d5c34421-6eee-49b3-a7a4-b79de4afeb0d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: Relat

In [9]:
print(nodes[0].relationships)
print(nodes[1].relationships)

{<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2fa6a6a0-3911-4743-aa53-a4e7032c5607', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='0af2325bc3976ef85024e35769344ac7138fe4c21700c2bf55d2109039882bb4'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='d5c34421-6eee-49b3-a7a4-b79de4afeb0d', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='524e34237455ee85fecf2724237a8e825e136e9dff359bbde44c4c3a656c1d95')}
{<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2fa6a6a0-3911-4743-aa53-a4e7032c5607', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='0af2325bc3976ef85024e35769344ac7138fe4c21700c2bf55d2109039882bb4'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='38081e25-27e8-4152-8da3-47627107ae55', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='edcd2c00a132ff9c5b9c159f7b6bcb26b00e8fc16d6df454d4f1c3639e01c294')}


Phần thiết lập mối quan hệ giữa các node các node này thường sẽ được tạo tự động, các bạn có thể thấy trong ví dụ trên, các mối quan hệ đã được tạo tự động. Ta có thể thấy node 1 có node SOURCE là doc, có mối quan hệ NEXT với node 2. Tương tự thì node 2 có mối quan hệ PREVIOUS với node 1. Nói chung  chúng ta chỉ cần biết là các mối quan hệ này rất quan trọng, giúp việc truy vấn chính xác hơn.

## Indexing

Sau khi bạn nhập dữ liệu vào hệ thống, LlamaIndex sẽ giúp bạn lập chỉ mục dữ liệu vào một cấu trúc dễ dàng truy xuất. Quá trình này thường bao gồm việc tạo ra các vector embeddings và lưu trữ chúng trong một cơ sở dữ liệu chuyên biệt gọi là vector store. Việc lập chỉ mục giúp tổ chức dữ liệu sao cho dễ dàng tìm kiếm và truy xuất sau này.

Llama Index hỗ trợ nhiều loại index khác nhau như SummaryIndex, VectorStoreIndex, TreeIndex, KnowledgeGraphIndex, tùy vào từng trường hợp mà chúng ta sẽ chọn loại phù hợp. Nhìn chung, các loại index này đều thực hiện các chức năng tạo index, thêm node mới, và truy vấn.

In [11]:
from llama_index.core import Document, SummaryIndex
from llama_index.core.node_parser import TokenTextSplitter

text = "Con mèo ú nằm ườn bên cửa sổ. Tôi muốn mình cũng được như thế."
doc = Document(
    text=text
    )

splitter = TokenTextSplitter(
    chunk_size=20,
    chunk_overlap=5,
    separator= " "
)
nodes = splitter.get_nodes_from_documents([doc])
index = SummaryIndex(nodes)
index2 = SummaryIndex.from_documents([doc])

print("index 1", index.index_struct)
print("index 2", index2.index_struct)

Metadata length (2) is close to chunk size (20). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
index 1 IndexList(index_id='e30eaa05-fa23-4691-aee0-202137c60a79', summary=None, nodes=['86f99abc-3dac-4f52-9e6f-bfd56bc9557e', 'b987d5bc-0e51-4d60-a03d-2ad0f5c30cd0', '7374ac68-e3ed-4283-b3b4-5b72c42bb1c5'])
index 2 IndexList(index_id='9f45e5a7-0942-4d7f-8dac-d5037c02313c', summary=None, nodes=['e74a7bee-125e-47c0-85fc-68922a6c39d8'])


In [12]:
index2 = SummaryIndex.from_documents([doc])

In [13]:
print(index.index_struct)

IndexList(index_id='e30eaa05-fa23-4691-aee0-202137c60a79', summary=None, nodes=['86f99abc-3dac-4f52-9e6f-bfd56bc9557e', 'b987d5bc-0e51-4d60-a03d-2ad0f5c30cd0', '7374ac68-e3ed-4283-b3b4-5b72c42bb1c5'])


Trong ví dụ trên chúng ta taọ index từ nodes sử dụng SummayIndex, chúng ta có thể xem cấu trúc của index qua index_struct. Vậy là index đã sẵn sàng để truy xuất, từ đầu đến giờ chúng ta tạo Doc, tạo Node, tạo Index cũng chỉ nhằm mục đích để truy xuất thông tin từ dữ liệu thôi phải không nào? Để truy xuất dữ liệu trong index, Llama Index hỗ trợ nhiều công cụ truy xuất khác nhau, nhưng đơn giản nhất ở đây là mình tạo đối tượng query_engine từ index.as_query_engine(), tức là chúng ta tạo một công cụ hỏi đáp từ index. Việc sử dụng cũng rất đơn giản chỉ cần gọi phương thức query_engine.query() cùng với tham số là nội dung câu hỏi.

In [14]:
query_engine = index.as_query_engine()
respone = query_engine.query("mèo ú đang làm gì?")
print(respone)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

Trong đoạn code trên, chúng ta gặp lỗi vì chưa thiết lập khóa API của OPEN AI nên không thể sử dụng query_engine. Bây giờ chúng ta hãy thêm khóa để sửa lỗi.

In [15]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
import openai

openai.api_key = "sk-proj-your-api-key"
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.2)


Trong đoạn code trên, chúng ta thiết lập api_key từ open api, ngoài ra chúng ta thiết lập mô hình LLM mà chúng ta sử dụng là gpt-4o-mini, với temperature là 0.2. Ở đây, temperature có giá trị từ 0 đến 2, giá trị càng cao thì câu trả lời càng sáng tạo, ngẫu nhiên. Khi giá trị thấp hơn hoặc bằng 0 thì kết quả cho ra sẽ cố định hơn, tức là chúng ta hỏi một câu hỏi nhiều lần thì kết quả trả lời đều giống nhau. Nói chung tùy mục đích sử dụng mà ta sẽ điều chỉnh con số hợp lý.

In [16]:
query_engine = index.as_query_engine()
respone = query_engine.query("mèo ú đang làm gì?")
print(respone)

Mèo ú đang nằm ườn bên cửa sổ.


Vậy là chúng ta đã tìm hiểu về các khái niệm cơ bản trong llama index và xây dựng thành công mô trình truy vấn RAG đơn giản. Dưới đây là tóm tắt các bước xây dựng mô hình RAG cơ bản:

1. Tải dữ liệu dưới dạng Documents
2. Phân tích tài liệu thành các nút mạch lạc
3. Xây dựng chỉ mục được tối ưu hóa từ Nút
4. Chạy truy vấn trên chỉ mục để truy xuất các Nút có liên quan
5. Tổng hợp phản hồi cuối cùng